# Incremental Collaborative Filtering (ICR) - Algorithm experiments

Movielens 100k data set: https://maxhalford.github.io/files/datasets/ml_100k.zip

In [1]:
import csv
from pycallgraph import PyCallGraph
from pycallgraph.output import GraphvizOutput

In [154]:
class myICF():
    def __init__(self):
        self.user_ratings = {} # Dict to store ratings
        self.user_meta = {} # Dict to cache users info on number of ratings and average ratings
        self.user_pair_meta = {} # Dict to cache info on factors calculated for pairs of users (B, C, D, sum of ratings to co-rated items)
        self.corr_threshold=0.65
        self.high_rating=4
   
    def _new_user(self, user, item, rating):
        # initialize new user
        self.user_ratings[user] = {} # initializes user in ratings dict
        self.user_meta[user] = {'q': 0, 'avg.rating': 0} # initializes user in meta dict, assign number of items user has rated and avg rating of user

        # initializes pairs of existing users with new user in user_pair_meta dict
        for u in self.user_meta.keys(): 
            if u == user:
                continue
            self.user_pair_meta[(u, user)] = {'B': 0, 'C': 0, 'D': 0}
            self.user_pair_meta[(u, user)]['sum.co_ratings'] = {u: 0, user: 0, 'n': 0}
            
    def _new_rating(self, user, item, rating):
        # Submission of a new rating
        q = self.user_meta[user]['q'] # gets number of items user has rated
        A_avg_rating = self.user_meta[user]['avg.rating']
        new_avg = round( ( rating/( q+1 ) ) + ( q/( q+1 ) )*A_avg_rating, 2 ) # calculates new avg rating for active user
        delta_avg = new_avg - A_avg_rating # difference of user's previous and current avg rating
        
        for userB in self.user_meta.keys():
            if userB == user:
                continue
                
            B_avg_rating = self.user_meta[userB]['avg.rating']
            
            if item in self.user_ratings[userB].keys():
                # User B has rated the item                
                A_sum_coratings, B_sum_coratings, n_coratings, key = self._update_get_coratings(user, userB, item, rating, new_rating=True)
                B_rating = self.user_ratings[userB][item]
                
                e = ( rating-new_avg )*( B_rating-B_avg_rating ) - delta_avg*( B_sum_coratings - n_coratings*B_avg_rating )
                f = ( rating-new_avg )**2 + n_coratings*delta_avg**2 - 2*delta_avg*( A_sum_coratings - n_coratings*A_avg_rating )
                g = ( B_rating-B_avg_rating )**2
            else:
                # User B had not rated the item
                A_sum_coratings, B_sum_coratings, n_coratings, key = self._get_coratings(user, userB)
                
                e = - delta_avg*( B_sum_coratings - n_coratings*B_avg_rating )
                f = n_coratings*delta_avg**2 - 2*delta_avg*( A_sum_coratings - n_coratings*A_avg_rating )
                g = 0
            
            for factor, increment in zip(['B', 'C', 'D'], [e, f, g]):
                self.user_pair_meta[key][factor] += increment

        self.user_ratings[user][item] = rating # updates rating given by user to item
        self.user_meta[user]['q'] += 1 # updates number of items user has rated
        self.user_meta[user]['avg.rating'] = new_avg # updates avg rating
        
        
    def _update_rating(self, user, item, rating):
        # Update of an existing rating
        delta_rating = rating - self.user_ratings[user][item] # difference of user's previous and current rating for item
        q = self.user_meta[user]['q'] # gets number of items user has rated
        A_avg_rating = self.user_meta[user]['avg.rating']
        new_avg = round( delta_rating/q + A_avg_rating, 2 ) # calculates new avg rating for active user
        delta_avg = new_avg - A_avg_rating # difference of user's previous and current avg rating
        
        for userB in self.user_meta.keys():
            if userB == user:
                continue
                            
            B_avg_rating = self.user_meta[userB]['avg.rating']
            
            if item in self.user_ratings[userB].keys():
                # User B has rated the item                
                A_sum_coratings, B_sum_coratings, n_coratings, key = self._update_get_coratings(user, userB, item, rating, new_rating=False)
                B_rating = self.user_ratings[userB][item]
                
                e = delta_rating*( B_rating-B_avg_rating ) - delta_avg*( B_sum_coratings - n_coratings*B_avg_rating )
                # calculus of f is not clear for both cases
                f = delta_rating**2 + 2*delta_rating*( rating-new_avg ) + n_coratings*delta_avg**2 - 2*delta_avg*( A_sum_coratings - n_coratings*A_avg_rating )
                g = 0
            else:
                # User B had not rated the item
                A_sum_coratings, B_sum_coratings, n_coratings, key = self._get_coratings(user, userB)
                
                e = - delta_avg*( B_sum_coratings - n_coratings*B_avg_rating )
                # calculus of f is not clear for both cases
                f = n_coratings*delta_avg**2 - 2*delta_avg*( A_sum_coratings - n_coratings*A_avg_rating )
                g = 0
                
            for factor, increment in zip(['B', 'C', 'D'], [e, f, g]):
                self.user_pair_meta[key][factor] += increment
                
        self.user_ratings[user][item] = rating # updates rating given by user to item
        self.user_meta[user]['q'] += 1 # updates number of items user has rated
        self.user_meta[user]['avg.rating'] = new_avg # updates avg rating
        
        
    def _get_coratings(self, userA, userB):
        if (userB, userA) in self.user_pair_meta.keys():
            key = (userB, userA)
        else:
            key = (userA, userB)
        A_sum_coratings = self.user_pair_meta[key]['sum.co_ratings'][userA]
        B_sum_coratings = self.user_pair_meta[key]['sum.co_ratings'][userB]
        n_coratings = self.user_pair_meta[key]['sum.co_ratings']['n']

        return A_sum_coratings, B_sum_coratings, n_coratings, key
        
    def _update_get_coratings(self, userA, userB, item, rating, new_rating=True):
        if (userB, userA) in self.user_pair_meta.keys():
            key = (userB, userA)
        else:
            key = (userA, userB)
        if new_rating:            
            self.user_pair_meta[key]['sum.co_ratings'][userA] += rating
            A_sum_coratings = self.user_pair_meta[key]['sum.co_ratings'][userA]
            self.user_pair_meta[key]['sum.co_ratings'][userB] += self.user_ratings[userB][item]
            B_sum_coratings = self.user_pair_meta[key]['sum.co_ratings'][userB]
            self.user_pair_meta[key]['sum.co_ratings']['n'] += 1
            n_coratings = self.user_pair_meta[key]['sum.co_ratings']['n']
        else:
            self.user_pair_meta[key]['sum.co_ratings'][userA] += (rating - self.user_ratings[userA][item])
            A_sum_coratings = self.user_pair_meta[key]['sum.co_ratings'][userA]
            B_sum_coratings = self.user_pair_meta[key]['sum.co_ratings'][userB]
            n_coratings = self.user_pair_meta[key]['sum.co_ratings']['n']
                
        return A_sum_coratings, B_sum_coratings, n_coratings, key
    
    def run(self, user, item, rating):
        # initialize new user
        if user not in self.user_meta.keys(): 
            self._new_user(user, item, rating)
            
        # Submission of a new rating
        if item not in self.user_ratings[user].keys(): 
            self._new_rating(user, item, rating)
            
        # Update of an existing rating
        else: 
            self._update_rating(user, item, rating)
            
    def recommend(self, user, n_recs=10):
        item_count = {}
        for userB in self.user_meta.keys():
            if userB == user:
                continue
            if (userB, user) in self.user_pair_meta.keys():
                key = (userB, user)
            else:
                key = (user, userB)
            B = self.user_pair_meta[key]['B']
            C = self.user_pair_meta[key]['C']
            D = self.user_pair_meta[key]['D']
            try:
                pearson_corr = B / ( ( C**(1/2) ) * ( D**(1/2) ) ) # sometimes, zero division happen
                if (pearson_corr > 1.1) or (pearson_corr < -1.1):
                    print(pearson_corr, user, userB)
                if pearson_corr >= self.corr_threshold: # sometimes, C and D are complex, thus they cannot be compared
                    for item in self.user_ratings[userB].keys():
                        if item in self.user_ratings[user].keys():
                            continue
                        if self.user_ratings[userB][item] >= self.high_rating:
                            item_count.setdefault(item, 0)
                            item_count[item] += 1
            except:
                continue
        sorted_item_count = sorted(item_count.items(), key=lambda x: -x[1])
        recommended_items = [i[0] for i in sorted_item_count[:n_recs]]
        return recommended_items 

In [155]:
def stream(filepath, delimiter, max_cases=500):
    with open(filepath, 'r') as csvf:
        #load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf, delimiter=delimiter)
        n=0
        for row in csvReader:
            if n == max_cases:
                break
            n+=1
            yield row['user'], row['item'], float(row['rating'])

In [156]:
with PyCallGraph(output=GraphvizOutput()):
    icf = myICF()
    for user, item, rating in stream(filepath='ml_100k.csv', delimiter='\t', max_cases=10000):    
        icf.run(user, item, rating)

![graphviz](pycallgraph.png)

In [157]:
icf.recommend(user='259')

1.3143603497860947 259 725


['56', '180', '64', '100', '15', '318', '204', '202', '127', '508']

In [158]:
for u in icf.user_meta.keys():
    icf.recommend(user=u) # |pearson| correlations greater than |1|

1.3143603497860947 259 725
-1.4598340521725681 851 102
1.1185470587609745 119 802
1.1338934190276797 119 772
-1.2483239258936305 594 671
-1.2039785103723157 594 684
1.5990940333348822 23 671
1.1338934190276797 23 772
1.4149996692790974 276 793
1.3569216516292244 276 172
1.1043992387174757 532 817
1.7618840316763393 532 738
-1.2126781251816645 532 691
1.176070475121329 532 839
-2.3799621920294443 532 912
1.5296627870780422 821 207
1.270848558954653 291 860
1.6130852934895417 291 118
1.1043992387174757 817 532
1.202900183380919 817 506
1.109267213794995 817 397
-2.7808975371924425 817 725
1.165978071386053 195 941
-1.1659780713860528 195 287
1.1659780713860528 195 671
1.164611685492488 195 779
-1.1659780713860526 195 837
-1.328126004637097 195 382
1.1477388766637522 195 661
1.3915193672058188 756 893
1.592833178095913 756 738
1.1182776085773174 756 35
1.1338934190276795 756 772
1.3915193672058188 893 756
1.1354613364904704 893 703
1.123679181183074 893 63
1.1771459913801752 893 772
-1.56